In [71]:
import tensorflow as tf
import numpy as np
from numpy import ndarray
from itertools import combinations
from typing import List, Iterable, Tuple, Dict
from collections import namedtuple

In [2]:
def get_sparse_mask_matrix(current_mask):
    def create_indices(mask):
        positive_el_number = 0
        indices = []
        for i, v in enumerate(mask):
            if v == 1:
                indices.append([i,positive_el_number])
                positive_el_number += 1
        return np.array(indices, dtype=np.int32)
    
    indices = tf.py_func(create_indices, [current_mask], tf.int32)
    number_of_remaining_elements = tf.cast(tf.divide(tf.size(indices),2), tf.int32)
    values = tf.py_func(lambda _indices: np.ones(len(_indices), dtype=np.int32),
                        [indices], tf.int32)
    shape = tf.stack([tf.size(current_mask),number_of_remaining_elements])
    mask_matrix = tf.sparse_to_dense(sparse_indices=indices, sparse_values=values, output_shape=shape)
    return mask_matrix


def _test_get_sparse_mask_matrix(curr_mask, expected_mask_matrix):
    mask = get_sparse_mask_matrix(curr_mask)
    init = tf.global_variables_initializer()
    with tf.Session() as session:
        session.run(init)
        result = session.run(mask)
    assert np.array_equal(result, expected_mask_matrix)
#     return num_remaining

test_current_mask = tf.constant([1,0,1,0,1], dtype=tf.int32)
expected_mask_matrix = np.array([[1,0,0],
                                 [0,0,0],
                                 [0,1,0],
                                 [0,0,0],
                                 [0,0,1]])

_test_get_sparse_mask_matrix(test_current_mask, expected_mask_matrix)

In [58]:
def reduce_input_columns_with_current_mask(input_rows, current_mask):
    """
    Construct a sparse matrix containing the input_rows with only the 'allowed' columns
    """
    inputs_mask_matrix = get_sparse_mask_matrix(current_mask)
    return tf.matmul(input_rows, inputs_mask_matrix)

def _test_reduce_input_columns_with_current_mask(test_input_rows, test_current_mask, expected_reduced_inputs):
    reduced_inputs = reduce_input_columns_with_current_mask(test_input_rows, test_current_mask)

    init = tf.global_variables_initializer()
    with tf.Session() as session:
        session.run(init)
        result = session.run(reduced_inputs)
    assert np.array_equal(result, expected_reduced_inputs)
#     return result

test_inputs = tf.constant([[1,2,1,2,1],
                           [1,2,0,2,1]])
test_current_mask = tf.constant([1,0,1,0,1], dtype=tf.int32)
expected_reduced_inputs = np.array([[1,1,1],
                                    [1,0,1]])
_test_reduce_input_columns_with_current_mask(test_inputs, test_current_mask, expected_reduced_inputs)

In [97]:
def are_group_subsets_frequent(prev_groups, group_idxs, group_size):
    return all([(frozenset(sub_group) in prev_groups) for sub_group in combinations(group_idxs, group_size-1)])


def get_mapped_groups_cube_idxs_and_new_els_count(groups_cube_idxs):
    new_els = sorted(set([i for idx in groups_cube_idxs for i in idx]))
    groups_cube_idxs_map = {idx: i for i, idx in enumerate(new_els)}
    mapped_groups_cube_idxs = np.array([[groups_cube_idxs_map[i] for i in idx] for idx in groups_cube_idxs], 
                                       dtype=np.int32)
    return mapped_groups_cube_idxs, len(new_els)


def get_possible_groups_cube_and_mask(prev_group_indices: ndarray, group_size: int, current_mask):
    """
    TODO: Fix for case when no group_size groups are possible
    """
    prev_groups = set(map(frozenset, prev_group_indices))
    possible_el_indices = set(prev_group_indices.flatten())
    possible_el_combinations = list(combinations(possible_el_indices, group_size))
    groups_cube_idxs = [group_idxs for group_idxs in possible_el_combinations 
                        if are_group_subsets_frequent(prev_groups, group_idxs, group_size)]
    groups_cube_values = np.array([1 for _ in range(len(groups_cube_idxs))], dtype=np.int32)
    new_possible_el_indices = set([i for idx in groups_cube_idxs for i in idx])
    if (len(new_possible_el_indices) == 0):
        raise ValueError('No possible elements')
    mask = [1 if i in new_possible_el_indices else 0 for i in range(len(current_mask))]
    mapped_groups_cube_idxs, new_tot_els = get_mapped_groups_cube_idxs_and_new_els_count(groups_cube_idxs)
    groups_cube_shape = np.array([new_tot_els for _ in range(group_size)], dtype=np.int32)
    return tf.sparse_to_dense(sparse_indices=mapped_groups_cube_idxs,
                              output_shape=groups_cube_shape,
                              sparse_values=groups_cube_values,), mask


def _test_get_possible_groups_cube_and_mask(prev_group_indices, group_size, current_mask, 
                                            expected_groups_cube, expected_mask):
    poss_groups, mask = get_possible_groups_cube_and_mask(prev_group_indices, group_size, current_mask)
    
    init = tf.global_variables_initializer()
    with tf.Session() as session:
        session.run(init)
        result = session.run(poss_groups)
    assert np.array_equal(result, expected_groups_cube)
    assert np.array_equal(mask, expected_mask)
#     return result, mask
    
prev_group_indices_2d = np.array([[2,3],[2,4],[3,4], [3,5]])
group_size = 3
current_mask = np.array([0,0,1,1,1,1])
expected_groups_cube = np.array([[[0,0,0],
                                  [0,0,1],
                                  [0,0,0]],
                                 [[0,0,0],
                                  [0,0,0],
                                  [0,0,0]],
                                 [[0,0,0],
                                  [0,0,0],
                                  [0,0,0]]])
expected_mask = np.array([0,0,1,1,1,0])
_test_get_possible_groups_cube_and_mask(prev_group_indices_2d, group_size, current_mask, 
                                        expected_groups_cube, expected_mask)

In [61]:
def get_inputs_t_multiplied_by_transpose_permutations(input_groups, group_size):
    # We have the masked vectorised input rows as a N+1 dimensional tensor
    # Now multiply by N - 1 tensors whose axes have been permuted (on all but 0th axis) 
    cross_multiplied_input_groups = input_groups
    dims_list = list(range(1, group_size + 1))
    perm_list = list(range(0, group_size))
    L = len(dims_list)
    for dim in dims_list[:-1]: #permutations(range(1,current_N + 1)):
        perm = [0] + [((i + dim + L - 1) % L) + 1 for i in dims_list]
        print(f"perm{perm}")
#         perm = [0 if i == -1 else perm[i] for i in range(-1, len(perm))]
        input_groups_perm = tf.transpose(input_groups, perm=perm)
        #TODO: Test broadcasting always works when using same number of input rows as unique elements
        cross_multiplied_input_groups = tf.multiply(cross_multiplied_input_groups, input_groups_perm)
    return cross_multiplied_input_groups

In [90]:
def get_next_mask_and_groups(gc):
    """
    TODO: use tf.tensordot to join the input rows to the 'possible groups' cube
    ----- The input elements axis becomes the new dimension of the 
    """

    print(f"Get_next_mask_and_groups for group size {gc['current_N']}")
    
    inputs_reduced = reduce_input_columns_with_current_mask(gc["input_rows"],
                                                            gc["curr_bin_mask"])
    
    possible_groups_cube, mask = get_possible_groups_cube_and_mask(gc["prev_group_indices"], 
                                                                   gc["current_N"], 
                                                                   gc["curr_bin_mask"])
    
    shape = [gc["num_input_rows"]] + [gc["num_remaining_els"] for _ in range(0,gc["current_N"])]
    number_of_elements = gc["num_remaining_els"]**(gc["current_N"]-1)
    expanded_inputs_t = tf.reshape(tf.tile(inputs_reduced, [1,number_of_elements]), shape)
    
    traspose_multiplied_input_groups = get_inputs_t_multiplied_by_transpose_permutations(expanded_inputs_t,
                                                                                         gc["current_N"])
    print(possible_groups_cube)
    input_groups_t = tf.multiply(traspose_multiplied_input_groups, possible_groups_cube)
    
    group_counts_t = tf.reduce_sum(input_groups_t, axis=0)

    frequent_groups_indices = tf.where(group_counts_t > 0)
    frequent_groups_counts = tf.gather_nd(group_counts_t, frequent_groups_indices)
    next_possible_groups_indices = tf.where(group_counts_t >= gc["min_support"])
    
    return frequent_groups_indices, frequent_groups_counts, next_possible_groups_indices, mask


def test_get_next_mask_and_groups(gc):
    with tf.Session() as session:
        init = tf.global_variables_initializer()
        session.run(init)
        frequent_groups_indices, frequent_groups_counts, next_possible_groups_indices, next_mask = \
            get_next_mask_and_groups(gc)
        group_counts_array = session.run([frequent_groups_indices,
                                          frequent_groups_counts,
                                          next_possible_groups_indices])
    return group_counts_array, next_mask
    
original_input_els = ['A','B','C','D','E']
vectorised_inputs_stack_2 = tf.Variable([[1,1,1,1,0],
                                         [1,1,0,1,1],
                                         [0,1,1,1,0]], tf.int32)
frequent_bin_mask_2 = np.array([1,1,1,1,0], dtype=np.int32)
prev_group_indices_2 = np.array([[0,1],[1,2],[2,3],[1,3],[0,3]], dtype=np.int32)

input_3_5 = {
    "current_N": 3,
    "prev_group_indices": prev_group_indices_2,
    "original_els": original_input_els, 
    "num_original_els": 5,
    "input_rows": vectorised_inputs_stack_2,
    "num_input_rows": 3,
    "curr_bin_mask": frequent_bin_mask_2,
    "num_remaining_els": 4,
    "curr_group_count_totals": [], 
    "min_support": 1
}

test_get_next_mask_and_groups(input_3_5)

Get_next_mask_and_groups for group size 3
perm[0, 2, 3, 1]
perm[0, 3, 1, 2]
Tensor("SparseToDense_136:0", shape=(4, 4, 4), dtype=int32)


([array([[0, 1, 3],
         [1, 2, 3]]), array([2, 2], dtype=int32), array([[0, 1, 3],
         [1, 2, 3]])], [1, 1, 1, 1, 0])

In [67]:
def get_input_collections_as_binary_arrays(input_collections: List[Iterable]) -> Tuple[List, List[ndarray]]:
    """
    TODO: Get binarised_input_collections with tensorflow?
    """
    # First get complete set of input elements
    all_input_elements = list(sorted(set([el for row in input_collections for el in row])))
    # For each input collection, create array with 1 for element exists or 0 for not exists 
    # (extension: count the number of elements - for use with multiple element counting version)
    binarised_input_collections = [tf.Variable([1 if el in row else 0 for el in all_input_elements], dtype=tf.int32)
                                   for row in input_collections]
    return (all_input_elements, binarised_input_collections,)

In [98]:
GroupCounts = namedtuple('GroupCounts', 'group, count')

class AprioriFrequentSets:
    def __init__(self, group_counts_list: List[GroupCounts], min_supp):
        self.group_counts_list = group_counts_list
        self.min_support = min_supp
    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False


def apriori(input_id_to_collections, min_support:int=0) -> AprioriFrequentSets:
    """
    TODO: handle case where input_rows is list type
    TODO: when input_rows is dict with ids, also collect all the ids which are in the frequent item groups
    """
    with tf.Session() as session:
        all_input_elements, vectorised_input_collections = get_input_collections_as_binary_arrays(
            input_id_to_collections.values()
        )
        original_els_count = len(all_input_elements)
        original_input_rows_count = len(input_id_to_collections.keys())

        number_of_unique_input_els = len(all_input_elements)

        vectorised_inputs_stack = tf.stack(vectorised_input_collections)

        single_el_occurances = tf.reduce_sum(vectorised_inputs_stack, axis=0)

        # Get single set groups as itemsets (in original formats)
        single_member_indices = tf.where(single_el_occurances >= 0)
        single_member_groups = tf.gather_nd(all_input_elements, single_member_indices)
        single_member_group_counts = tf.gather_nd(single_el_occurances, single_member_indices)
        
        next_el_indices = tf.where(single_el_occurances >= min_support)

        frequent_single_bin_mask = tf.cast(single_el_occurances >= min_support, tf.int32)
        
        init = tf.global_variables_initializer()
        session.run(init)
        
        #Add these to the group counts total
        single_mem_indices, single_mem_groups, single_mem_counts, single_el_mask, single_el_indices = \
            session.run([single_member_indices, 
                         single_member_groups, 
                         single_member_group_counts, 
                         frequent_single_bin_mask,
                         next_el_indices])
    
        # TODO: add as frequent groups single_mem_indices

        gc = {
            "current_N": 1,
            "prev_group_indices": single_el_indices,
            "original_els": all_input_elements, 
            "num_original_els": original_els_count,
            "input_rows": vectorised_input_collections,
            "num_input_rows": original_input_rows_count,
            "curr_bin_mask": single_el_mask,
            "num_remaining_els": len(single_el_indices),
            "min_support": min_support
        }
        for dim in range(2,5):
            gc["current_N"] = dim
            
            try:
                frequent_groups_indices, frequent_groups_counts, next_possible_groups_indices, next_mask = \
                    get_next_mask_and_groups(gc)
            except ValueError as err:
                break
                
            group_counts_array = session.run([frequent_groups_indices,
                                          frequent_groups_counts,
                                          next_possible_groups_indices])
            print(group_counts_array)
            gc["prev_group_indices"] = group_counts_array[2]
            gc["curr_bin_mask"] = next_mask
            gc["num_remaining_els"] = np.sum(next_mask)
            #TODO: convert group_counts_array elements to GroupCounts objects and collect in some list
        return group_counts_array, next_mask
        
def _test_apriori(input_rows: Dict, min_support, expected_apriori_frequent_sets: AprioriFrequentSets):
    result = apriori(input_rows, min_support)
    
#     assert expected_apriori_frequent_sets == result
    return result


input_dict_1 = {0:['A','B','C'],
                1:['A','B','C'],
                2:['A','B','D'],
                3:['A','B','D'],
                4:['A','B','E']}
min_support_1 = 2
group_counts_list_1 = [GroupCounts({'A'},5),GroupCounts({'B'},5),GroupCounts({'C'},2),GroupCounts({'D'},2),
                       GroupCounts({'E'},1),
                       GroupCounts({'A','B'},5),GroupCounts({'A','C'},2),GroupCounts({'A','D'},2),
                       GroupCounts({'B','C'},2),GroupCounts({'B','D'},2),
                       GroupCounts({'A','B','C'},2),GroupCounts({'A','B','D'},2)]
expected_apriori_frequent_sets_1 = AprioriFrequentSets(group_counts_list_1, min_support_1)

_test_apriori(input_dict_1, min_support_1, expected_apriori_frequent_sets_1)

Get_next_mask_and_groups for group size 2
perm[0, 2, 1]
Tensor("SparseToDense_153:0", shape=(4, 4), dtype=int32)
[array([[0, 1],
       [0, 2],
       [0, 3],
       [1, 2],
       [1, 3]]), array([5, 2, 2, 2, 2], dtype=int32), array([[0, 1],
       [0, 2],
       [0, 3],
       [1, 2],
       [1, 3]])]
Get_next_mask_and_groups for group size 3
perm[0, 2, 3, 1]
perm[0, 3, 1, 2]
Tensor("SparseToDense_155:0", shape=(4, 4, 4), dtype=int32)
[array([[0, 1, 2],
       [0, 1, 3]]), array([2, 2], dtype=int32), array([[0, 1, 2],
       [0, 1, 3]])]
Get_next_mask_and_groups for group size 4


([array([[0, 1, 2],
         [0, 1, 3]]), array([2, 2], dtype=int32), array([[0, 1, 2],
         [0, 1, 3]])], [1, 1, 1, 1, 0])